# <b> <p align="center"> <span style="color: #DCC43C"> GOING MODULER <span> </p> </b>
### <b> <p align="center"> <span style="color: #BFF0FF"> script mode <span> </p> </b>

## What is script mode?

**Script mode** uses [Jupyter Notebook cell magic](https://ipython.readthedocs.io/en/stable/interactive/magics.html) (special commands) to turn specific cells into Python scripts.

For example if you run the following code in a cell, you'll create a Python file called `hello_world.py`:

```
%%writefile hello_world.py
print("hello world, machine learning is fun!")
```

You could then run this Python file on the command line with:

```
python hello_world.py

>>> hello world, machine learning is fun!
```

The main cell magic we're interested in using is `%%writefile`.

Putting `%%writefile filename` at the top of a cell in Jupyter or Google Colab will write the contents of that cell to a specified `filename`.

> **Question:** Do I have to create Python files like this? Can't I just start directly with a Python file and skip using a Google Colab notebook?
>
> **Answer:** Yes. This is only *one* way of creating Python scripts. If you know the kind of script you'd like to write, you could start writing it straight away. But since using Jupyter/Google Colab notebooks is a popular way of starting off data science and machine learning projects, knowing about the `%%writefile` magic command is a handy tip.

## What has script mode got to do with PyTorch?

If you've written some useful code in a Jupyter Notebook or Google Colab notebook, chances are you'll want to use that code again.

And turning your useful cells into Python scripts (`.py` files) means you can use specific pieces of your code in other projects.

This practice is not PyTorch specific.

But it's how you'll see many different online PyTorch repositories structured.

### PyTorch in the wild

For example, if you find a PyTorch project on GitHub, it may be structured in the following way:

```
pytorch_project/
├── pytorch_project/
│   ├── data_setup.py
│   ├── engine.py
│   ├── model.py
│   ├── train.py
│   └── utils.py
├── models/
│   ├── model_1.pth
│   └── model_2.pth
└── data/
    ├── data_folder_1/
    └── data_folder_2/
```

Here, the top level directory is called `pytorch_project` but you could call it whatever you want.

Inside there's another directory called `pytorch_project` which contains several `.py` files, the purposes of these may be:
* `data_setup.py` - a file to prepare data (and download data if needed).
* `engine.py` - a file containing various training functions.
* `model_builder.py` or `model.py` - a file to create a PyTorch model.
* `train.py` - a file to leverage all other files and train a target PyTorch model.
* `utils.py` - a file dedicated to helpful utility functions.

And the `models` and `data` directories could hold PyTorch models and data files respectively (though due to the size of models and data files, it's unlikely you'll find the *full* versions of these on GitHub, these directories are present above mainly for demonstration purposes).

> **Note:** There are many different ways to structure a Python project and subsequently a PyTorch project. This isn't a guide on *how* to structure your projects, only an example of how you *might* come across PyTorch projects in the wild. For more on structuring Python projects, see Real Python's [*Python Application Layouts: A Reference*](https://realpython.com/python-application-layouts/) guide.

## What we're going to cover

By the end of this notebook you should finish with a directory structure of:

```
going_modular/
├── going_modular/
│   ├── data_setup.py
│   ├── engine.py
│   ├── model_builder.py
│   ├── train.py
│   └── utils.py
├── models/
│   ├── 05_going_modular_cell_mode_tinyvgg_model.pth
│   └── 05_going_modular_script_mode_tinyvgg_model.pth
└── data/
    └── pizza_steak_sushi/
        ├── train/
        │   ├── pizza/
        │   │   ├── image01.jpeg
        │   │   └── ...
        │   ├── steak/
        │   └── sushi/
        └── test/
            ├── pizza/
            ├── steak/
            └── sushi/
```

Using this directory structure, you should be able to train a model from within a notebook with the command:

```
!python going_modular/train.py
```

Or from the command line with:

```
python going_modular/train.py
```

In essence, we will have turned our helpful notebook code into **reusable modular code**.

## 0. Creating a folder for storing Python scripts

Since we're going to be creating Python scripts out of our most useful code cells, let's create a folder for storing those scripts.

We'll call the folder `going_modular` and create it using Python's [`os.makedirs()`](https://docs.python.org/3/library/os.html) method.

In [1]:
import os 
import torch
from torch import nn

os.makedirs("going_moduler",exist_ok=True)

## 1. Get data

We're going to start by downloading the same data we used in [notebook 04](https://github.com/ShafaetUllah032/DL_with_PyTorch/blob/main/04%20pytorch%20custom%20dataset.ipynb), the `pizza_steak_sushi` dataset with images of pizza, steak and sushi.

In [2]:
# from google.oauth2 import service_account
# from googleapiclient.discovery import build
# import requests
# import os

# # Path to the credentials.json file you downloaded
# credentials_path = 'credentials.json'

# # Authenticate using the service account
# SCOPES = ['https://www.googleapis.com/auth/drive.readonly']
# creds = service_account.Credentials.from_service_account_file(credentials_path, scopes=SCOPES)
# service = build('drive', 'v3', credentials=creds)

# # Folder ID from the shareable link
# folder_id = 'YOUR_FOLDER_ID_HERE'

# # List files in the folder
# results = service.files().list(q=f"'{folder_id}' in parents", fields="files(id, name)").execute()
# files = results.get('files', [])

# if not files:
#     print('No files found.')
# else:
#     print('Files:')
#     for file in files:
#         print(f"{file['name']} ({file['id']})")

#     # Create a folder to save the downloaded files
#     download_folder = 'downloaded_images'
#     os.makedirs(download_folder, exist_ok=True)

#     # Download each file
#     for file in files:
#         file_id = file['id']
#         file_name = file['name']
#         request = service.files().get_media(fileId=file_id)
#         file_path = os.path.join(download_folder, file_name)

#         with open(file_path, 'wb') as f:
#             downloader = requests.get(f"https://www.googleapis.com/drive/v3/files/{file_id}?alt=media", headers={"Authorization": f"Bearer {creds.token}"})
#             f.write(downloader.content)
#             print(f"Downloaded {file_name}")


In [3]:
import os
import zipfile
from pathlib import Path
import requests

# setup path to data folder

data_path=Path("data/")
image_path=data_path/"pizza_steak_sushi"


# If the imamge floder doesn't exist , download it and prepare it

if image_path.is_dir():
    print(f"{image_path} exist no need to create ..")
else:
    image_path.mkdir(parents=True,exist_ok=True)
    print(f"{image_path} created successfully")


# Downlod pizza, steak, sushi data

if Path(data_path/"pizza_steak_sushi").is_dir:
    print("folder exists, skipping downloading")

else:
    with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
        request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
        print("Downloading pizza, steak, sushi data...")
        f.write(request.content)

    # Unzip pizza, steak, sushi data
    with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
        print("Unzipping pizza, steak, sushi data...")
        zip_ref.extractall(image_path)

    # Remove zip file
    os.remove(data_path / "pizza_steak_sushi.zip")
    print("zip file removed ... ")

data\pizza_steak_sushi exist no need to create ..
folder exists, skipping downloading


In [4]:
# Setup train and testing paths

train_dir=image_path/"train" 
test_dir=image_path/"test"

train_dir,test_dir

(WindowsPath('data/pizza_steak_sushi/train'),
 WindowsPath('data/pizza_steak_sushi/test'))

## 2. Create Datasets and DataLoaders

Let's turn our data into PyTorch `Dataset`'s and `DataLoader`'s and find out a few useful attributes from them such as `classes` and their lengths.

In [5]:
from torchvision import datasets, transforms

# Create simple transform

data_transform=transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor(),
])

# Use ImageFolder to create dataset(s)
train_data = datasets.ImageFolder(root=train_dir, # target folder of images
                                  transform=data_transform, # transforms to perform on data (images)
                                  target_transform=None) # transforms to perform on labels (if necessary)

test_data = datasets.ImageFolder(root=test_dir,
                                 transform=data_transform)

print(f"Train data:\n{train_data}\nTest data:\n{test_data}")

Train data:
Dataset ImageFolder
    Number of datapoints: 225
    Root location: data\pizza_steak_sushi\train
    StandardTransform
Transform: Compose(
               Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )
Test data:
Dataset ImageFolder
    Number of datapoints: 75
    Root location: data\pizza_steak_sushi\test
    StandardTransform
Transform: Compose(
               Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )


In [6]:
# Get class names as a list
class_names = train_data.classes
class_names

['pizza', 'steak', 'sushi']

In [7]:
# Can also get class names as a dict
class_dict = train_data.class_to_idx
class_dict

{'pizza': 0, 'steak': 1, 'sushi': 2}

In [8]:
# Check the lengths
len(train_data), len(test_data)

(225, 75)

In [9]:
# Turn train and test Datasets into DataLoaders
from torch.utils.data import DataLoader

train_dataloader = DataLoader(dataset=train_data,
                              batch_size=1, # how many samples per batch?
                              num_workers=1, # how many subprocesses to use for data loading? (higher = more)
                              shuffle=True) # shuffle the data?

test_dataloader = DataLoader(dataset=test_data,
                             batch_size=1,
                             num_workers=1,
                             shuffle=False) # don't usually need to shuffle testing data

train_dataloader, test_dataloader

(<torch.utils.data.dataloader.DataLoader at 0x1986dab8eb0>,
 <torch.utils.data.dataloader.DataLoader at 0x198127620a0>)

In [10]:
# Check out single image size/shape
img, label = next(iter(train_dataloader))

# Batch size will now be 1, try changing the batch_size parameter above and see what happens
print(f"Image shape: {img.shape} -> [batch_size, color_channels, height, width]")
print(f"Label shape: {label.shape}")

Image shape: torch.Size([1, 3, 64, 64]) -> [batch_size, color_channels, height, width]
Label shape: torch.Size([1])


### 2.1 Create Datasets and DataLoaders (script mode)

Rather than rewriting all of the code above everytime we wanted to load data, we can turn it into a script called `data_setup.py`.

Let's capture all of the above functionality into a function called `create_dataloaders()`.

In [11]:
%%writefile going_moduler/data_setup.py

"""
contains functionality for creating pytorch Dataloader for
image classification data.

"""
import os 
from torch.utils.data import  dataloader
from torchvision import datasets,transforms  


NUM_WORKER=os.cpu_count()

def create_dataloader(
    train_dir: str,
    test_dir:  str,
    transform:  transforms.Compose,
    batch_size: int,
    num_workers: int=NUM_WORKER):
    
    """ Creates training and testing Dataloaders 

    Takes in a training directories and testing directories path and turns then into 
    pytorch Datasets and then into pytorch DataLoaders.

    Args:
        train_dir: path to training directory
        test_dir: path to testing directory
        transform: torchvision transforms to perform on training and testing data
        batch_size: Number of samples per batch in each of the DataLoaders
        num_workers: An integer for number of workers per dataLoader.

    Returns:
        A tuple of (triain_dataloader,test_dataloader,class_names)
        where class_name is a list of the target classes.
        Example usages:
        train_dataloader,test_dataloader,class_names=\
            =create_dataloader(train_dir:path/to/train_dir,
                            test_dir: path/to/test_dri,
                            transform=some_transform,
                            batch_size=32,
                            num_workers=4)

    """

    # Use ImageFolder to create dataset(s)

    train_data=datasets.ImageFolder(root=train_dir,
                                    transform=transform)

    test_data=datasets.ImageFolder(root=test_dir,
                                transform=transforms)
                            
    # Get class names
    class_names=train_data.classes


    # Turn Image into dataloader

    train_dataloader=DataLoader(dataset=train_data,
                                batch_size=batch_size,
                                shuffle=True,
                                num_worker=NUM_WORKER,
                                pin_memory=True)

    test_dataloader=DataLoader(dataset=test_data,
                            batch_size=batch_size,
                            shuffle=True,
                            num_worker=NUM_WORKER,
                            pin_number=True)

    return train_dataloader,test_dataloader,class_names





Overwriting going_moduler/data_setup.py


## 3. Making a model (TinyVGG)

We're going to use the same model we used in notebook 04: TinyVGG from the CNN Explainer website.

The only change here from notebook 04 is that a docstring has been added using [Google's Style Guide for Python](https://google.github.io/styleguide/pyguide.html#384-classes).

In [12]:
import torch
from torch import nn

class TinyVGG(nn.Module):
    """ 
    Create the TinyVGG architecture.

    Replicates the TinyVGG architecture from the CNN-explainer website in PyTorch.
    see the original architecture here: https://poloclub.github.io/cnn-explainer/

    Args:
        input_shape  :  An integer indicating number of input channels.
        hidden_unit  :  An integer indicating number of hidden units.
        output_shape :  An integer indicating number of output units.
    
    Return:
        None
     
    """
    
    def __init__(self,
                 input_shape:  int,
                 hidden_units:  int,
                 output_shape: int ):
        
        super().__init__()

        self.conv_block_1=nn.Sequential(
            nn.Conv2d(in_channels=input_shape,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=2)          
        )
        self.conv_block_2=nn.Sequential(
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      stride=1,
                      kernel_size=3,
                      padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=2)
        )

        self.classifier=nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units*13*13,
                      out_features=output_shape)
        )
    
    def forward(self,x: torch.tensor):
        x=self.classifier(self.conv_block_2(self.conv_block_1(x)))
        return x


Now let's create an instance of `TinyVGG` and put it on the target device.

> **Note:** If you're using Google Colab, and you'd like to use a GPU (recommended), you can turn one on via going to Runtime -> Change runtime type -> Hardware accelerator -> GPU.

In [13]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

# Instantiate an instance of the model
torch.manual_seed(42)
model_0 = TinyVGG(input_shape=3, # number of color channels (3 for RGB)
                  hidden_units=10,
                  output_shape=len(train_data.classes)).to(device)
model_0

TinyVGG(
  (conv_block_1): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1690, out_features=3, bias=True)
  )
)

Let's check out our model by doing a dummy forward pass.

In [14]:
model_0.state_dict()

OrderedDict([('conv_block_1.0.weight',
              tensor([[[[ 0.1471,  0.1597, -0.0451],
                        [ 0.1768, -0.0422,  0.0388],
                        [-0.0937,  0.1130,  0.1697]],
              
                       [[-0.1412,  0.1673,  0.0360],
                        [ 0.1422,  0.0261,  0.0928],
                        [-0.0272,  0.1484,  0.0284]],
              
                       [[-0.0898,  0.0491, -0.0887],
                        [-0.0226, -0.0782,  0.1277],
                        [-0.1519, -0.0887, -0.0543]]],
              
              
                      [[[-0.1157,  0.0182, -0.1901],
                        [ 0.1738, -0.1635,  0.1486],
                        [ 0.0320, -0.0625,  0.1189]],
              
                       [[ 0.0300,  0.1555,  0.0210],
                        [-0.0607,  0.0517, -0.0522],
                        [ 0.0810,  0.1718,  0.1112]],
              
                       [[-0.0841,  0.1111,  0.0344],
                 

In [15]:
loss_fn=torch.nn.CrossEntropyLoss()

In [16]:
# 1. Get a batch of images and labels from the DataLoader
img_batch, label_batch = next(iter(train_dataloader))

# 2. Get a single image from the batch and unsqueeze the image so its shape fits the model
img_single, label_single = img_batch[0].unsqueeze(dim=0), label_batch[0]
print(f"Single image shape: {img_single.shape}\n")
# print(f"imgage {img}")

# 3. Perform a forward pass on a single image
model_0.eval()
with torch.inference_mode():
    pred = model_0(img_single.to(device))
    loss=loss_fn(pred,label_single.unsqueeze(dim=0).to(device))

# 4. Print out what's happening and convert model logits -> pred probs -> pred label
print(f"Output logits:\n{pred}\n")
print(f"Output the loss value... for {label_single.unsqueeze(0)} is: {loss}")
print(f"Output prediction probabilities:\n{torch.softmax(pred, dim=1)}\n") # using softmax , dim=1 we convert the each row sum of 1 
print(f"Output prediction label:\n{torch.argmax(torch.softmax(pred, dim=1), dim=1)}\n") # using argmax , dim=1 , select the index of maximum value of each row
print(f"Actual label:\n{label_single}")

Single image shape: torch.Size([1, 3, 64, 64])

Output logits:
tensor([[ 0.0202, -0.0020,  0.0101]], device='cuda:0')

Output the loss value... for tensor([0]) is: 1.0878868103027344
Output prediction probabilities:
tensor([[0.3369, 0.3295, 0.3335]], device='cuda:0')

Output prediction label:
tensor([0], device='cuda:0')

Actual label:
0


## 4. Creating `train_step()` and `test_step()` functions and `train()` to combine them  

Rather than writing them again, we can reuse the `train_step()` and `test_step()` functions from [notebook 04](https://github.com/ShafaetUllah032/DL_with_PyTorch/blob/main/04%20pytorch%20custom%20dataset.ipynb).

The same goes for the `train()` function we created.

The only difference here is that these functions have had docstrings added to them in [Google's Python Functions and Methods Style Guide](https://google.github.io/styleguide/pyguide.html#383-functions-and-methods).

In [17]:
from typing import Tuple

def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
    """Trains a PyTorch model for a single epoch.

    Turns a target PyTorch model to training mode and then
    runs through all of the required training steps (forward
    pass, loss calculation, optimizer step).

    Args:
    model: A PyTorch model to be trained.
    dataloader: A DataLoader instance for the model to be trained on.
    loss_fn: A PyTorch loss function to minimize.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
    A tuple of training loss and training accuracy metrics.
    In the form (train_loss, train_accuracy). For example:

    (0.1112, 0.8743)
    """
    # Put model in train mode
    model.train()

    # Setup train loss and train accuracy values
    train_loss, train_acc = 0, 0

    # Loop through data loader data batches
    for batch, (X, y) in enumerate(dataloader):
        # Send data to target device
        X, y = X.to(device), y.to(device)

        # 1. Forward pass
        y_pred = model(X)

        # 2. Calculate  and accumulate loss
        loss = loss_fn(y_pred, y)
        print(f" batch no: {batch} , loss : {loss}")
        train_loss += loss.item()

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

        # Calculate and accumulate accuracy metric across all batches
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item()/len(y_pred)

    # Adjust metrics to get average loss and accuracy per batch
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc

Now we'll do `test_step()`.

In [18]:
def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
    """Tests a PyTorch model for a single epoch.

    Turns a target PyTorch model to "eval" mode and then performs
    a forward pass on a testing dataset.

    Args:
    model: A PyTorch model to be tested.
    dataloader: A DataLoader instance for the model to be tested on.
    loss_fn: A PyTorch loss function to calculate loss on the test data.
    device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
    A tuple of testing loss and testing accuracy metrics.
    In the form (test_loss, test_accuracy). For example:

    (0.0223, 0.8985)
    """
    # Put model in eval mode
    model.eval()

    # Setup test loss and test accuracy values
    test_loss, test_acc = 0, 0

    # Turn on inference context manager
    with torch.inference_mode():
        # Loop through DataLoader batches
        for batch, (X, y) in enumerate(dataloader):
            # Send data to target device
            X, y = X.to(device), y.to(device)

            # 1. Forward pass
            test_pred_logits = model(X)

            # 2. Calculate and accumulate loss
            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()

            # Calculate and accumulate accuracy
            test_pred_labels = test_pred_logits.argmax(dim=1)
            test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

    # Adjust metrics to get average loss and accuracy per batch
    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc

In [19]:
optimizer=torch.optim.SGD(params=model_0.parameters(),
                                lr=0.01)

And we'll combine `train_step()` and `test_step()` into `train()`.

In [20]:
from typing import Dict, List

from tqdm.auto import tqdm

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List[float]]:
    """Trains and tests a PyTorch model.

    Passes a target PyTorch models through train_step() and test_step()
    functions for a number of epochs, training and testing the model
    in the same epoch loop.

    Calculates, prints and stores evaluation metrics throughout.

    Args:
    model: A PyTorch model to be trained and tested.
    train_dataloader: A DataLoader instance for the model to be trained on.
    test_dataloader: A DataLoader instance for the model to be tested on.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    loss_fn: A PyTorch loss function to calculate loss on both datasets.
    epochs: An integer indicating how many epochs to train for.
    device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
    A dictionary of training and testing loss as well as training and
    testing accuracy metrics. Each metric has a value in a list for
    each epoch.
    In the form: {train_loss: [...],
                  train_acc: [...],
                  test_loss: [...],
                  test_acc: [...]}
    For example if training for epochs=2:
                 {train_loss: [2.0616, 1.0537],
                  train_acc: [0.3945, 0.3945],
                  test_loss: [1.2641, 1.5706],
                  test_acc: [0.3400, 0.2973]}
    """
    # Create empty results dictionary
    results = {"train_loss": [],
      "train_acc": [],
      "test_loss": [],
      "test_acc": []
    }

    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
        test_loss, test_acc = test_step(model=model,
          dataloader=test_dataloader,
          loss_fn=loss_fn,
          device=device)

        # Print out what's happening
        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    # Return the filled results at the end of the epochs
    return results

### 4.1 Creating `train_step()` and `test_step()` functions and `train()` to combine them (script mode)   

To create a script for `train_step()`, `test_step()` and `train()`, we'll combine their code all into a single cell.

We'll then write that cell to a file called `engine.py` because these functions will be the "engine" of our training pipeline.

We can do so with the magic line `%%writefile going_modular/engine.py`.

We'll also make sure to put all the imports we need (`torch`, `typing`, and `tqdm`) at the top of the cell.

In [21]:

""" 
Contains functionfor training and testing a pytorch model
"""
from typing import Dict,List,Tuple
import torch
from tqdm.auto import tqdm
import torch.utils.data

def train_step(model : torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn : torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device)-> Tuple(float,float):
    
    """ Trains a pyotrch model for single epoch.

    Turns a target pytorch model to training mode and then
    runs through all of the required training steps(forward,
    calculate the loss function , optimize the zero grad, step 
    backward, optimizer step)

    Args:
        model      :     A pytorch model to be trained
        dataloader :     A Dataloader instance for the model to be trained on.
        loss_fn    :     A pytorch loss function to minimize
        optimizer  :     A pytorch optimizer to help minimize the loss function
        device     :     A target device to compute on (e.g. "cuda" or "cup")
    
    Returns:
        A tuple of training loss and training accuracy metrics.
        In the form  (train_loss, train_accuacy). For example:

        (0.1232,0.4145)
    """

    # put model in train mode

    model.train()

    # Setup train loss and train accuracy values

    train_loss,train_acc =0,0

    # Loop through the batch and dataloader

    for batch, (X,y) in enumerate(dataloader):

        # send data to target device

        X,y=X.to(device) , y.to(device)

        # 1. do forward pass 
        y_pred = model(X)

        # 2. calculate the loss function
        loss=loss_fn(y_pred,y)
        train_loss += loss

        # 3. opitimize the zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. optimizer step

        optimizer.ste()

        # Calculate the accumulate accuracy metric across all batches

        y_pred_class=torch.argmax(torch.softmax(y_pred,dim=1),dim=1)
        train_acc += (y_pred_class == y).sum().item()/len(y_pred)
    
    # Adjust metrics to get average loss an accuracy per batch

    train_loss=train_loss/len(data)
    train_acc=train_acc/len(data)

    return train_loss,train_acc


def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.dataloader,
              loss_fn: loss_fn,
              device:torch.device) -> Tuple(float,float):
    """Test a pytorch model for a single epoch, 
    
    Turns a target Pytorch model to "eval" mode and
    then performs a forward pass on a testing dataset.

    Args:
        model      :  A pytorch model to be tested
        dataloader :  A DataLoader instance for the model to be tasted on.
        loss_fn    :  A pytorch loss function to calculate the loss on the test data
        device     :  A target device to compute on (e.g. "cuda" or "cpu")

    Returns:
        A tuple of testing loss and testing accuracy metrics.
        In the form (test_loss,test_accuracy). For example:

        (0.2354,0.2523)
    """
    # put the model in eval mode
    model.eval()

    # Setup test loss and test_accuracy 

    test_loss,test_acc=0,0

    # Setup testing loss and test accuacy values
    with torch.inference_mode():
        # Loop through DataLoader batches
        for batch,(X,y) in enumerates(dataloader):
            # send data to traget device
            X,y =X.to(device),y.to(device)

            # 1. Forward pass
            test_pred_logits = mode(x)

            # 2. Calculate the loss function
            loss=loss_fn(test_pred_logits,y)
            test_loss +=loss

            # 3. Calculate and accumulate accuracy
            test_pred_labels=torch.argmax(torch.softmax(test_pred_logits,dim=1),dim=1)
            test_acc += (test_pred_class==y).sum().item()/len(test_pred_labels)

        
    # Adjust metrics to get average loss and accuracy per batch
    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc 


def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List[float]]:
    """Trains and tests a PyTorch model.

    Passes a target PyTorch models through train_step() and test_step()
    functions for a number of epochs, training and testing the model
    in the same epoch loop.

    Calculates, prints and stores evaluation metrics throughout.

    Args:
    model: A PyTorch model to be trained and tested.
    train_dataloader: A DataLoader instance for the model to be trained on.
    test_dataloader: A DataLoader instance for the model to be tested on.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    loss_fn: A PyTorch loss function to calculate loss on both datasets.
    epochs: An integer indicating how many epochs to train for.
    device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
    A dictionary of training and testing loss as well as training and
    testing accuracy metrics. Each metric has a value in a list for
    each epoch.
    In the form: {train_loss: [...],
              train_acc: [...],
              test_loss: [...],
              test_acc: [...]}
    For example if training for epochs=2:
             {train_loss: [2.0616, 1.0537],
              train_acc: [0.3945, 0.3945],
              test_loss: [1.2641, 1.5706],
              test_acc: [0.3400, 0.2973]}
    """
    # Create empty results dictionary
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
    }

    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
        test_loss, test_acc = test_step(model=model,
          dataloader=test_dataloader,
          loss_fn=loss_fn,
          device=device)

        # Print out what's happening
        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    # Return the filled results at the end of the epochs
    return results


TypeError: Type Tuple cannot be instantiated; use tuple() instead

## 5. Creating a function to save the model

Let's setup a function to save our model to a directory.

In [ ]:
from pathlib import Path

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):
    """Saves a PyTorch model to a target directory.

    Args:
    model: A target PyTorch model to save.
    target_dir: A directory for saving the model to.
    model_name: A filename for the saved model. Should include
      either ".pth" or ".pt" as the file extension.

    Example usage:
    save_model(model=model_0,
               target_dir="models",
               model_name="05_going_modular_tingvgg_model.pth")
    """
    # Create target directory
    target_dir_path = Path(target_dir)
    target_dir_path.mkdir(parents=True,
                        exist_ok=True)

    # Create model save path
    assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
    model_save_path = target_dir_path / model_name

    # Save the model state_dict()
    print(f"[INFO] Saving model to: {model_save_path}")
    torch.save(obj=model.state_dict(),
             f=model_save_path)

### 5.1 Creating a function to save the model (script mode)

How about we add our `save_model()` function to a script called `utils.py` which is short for "utilities".

We can do so with the magic line `%%writefile going_modular/utils.py`.

In [ ]:
%%writefile going_moduler/utils.py
"""
Contains various utility functions for Pytorch model training and saving
"""
from pathlib import Path
import torch

def save_model(model:torch.nn.Module,
               target_dir: str,
               model_name: str):
    """Saves a pytorch model to a target directory.

    Args:
        model      : A target pytorch model to save.
        target_dir : A directory for saving the model to.
        model_name : A filename for the saved model. should include either ".pth" or
        ".pt" as the file extenstion.
    
    Example usages:
    save_model(model_0,
               target_dir="models",
               model_name="05_going_modular_tingvgg_model.pth")
    """
    # Create target directory
    target_dir_path = Path(train_dir)
    train_dir_path.mkdir(parents=True,
                         exist_ok=True)

    # Create a model save path
    assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth' "
    model_save_path=target_dir_path/model_name

    # Save the model  state_dict()
    print(f"[INFO] Saving model to : {mdoel_save_path}")
    torch.save(obj=model_state_dict(),
               f=model_save_path)

Overwriting going_moduler/utils.py


## 6. Train, evaluate and save the model

Let's leverage the functions we've got above to train, test and save a model to file.

In [ ]:
# Set random seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Set number of epochs
NUM_EPOCHS = 5

# Recreate an instance of TinyVGG
model_0 = TinyVGG(input_shape=3, # number of color channels (3 for RGB)
                  hidden_units=10,
                  output_shape=len(train_data.classes)).to(device)

# Setup loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model_0.parameters(), lr=0.001)

# Start the timer
from timeit import default_timer as timer
start_time = timer()

# Train model_0
model_0_results = train(model=model_0,
                        train_dataloader=train_dataloader,
                        test_dataloader=test_dataloader,
                        optimizer=optimizer,
                        loss_fn=loss_fn,
                        epochs=NUM_EPOCHS,
                        device=device)

# End the timer and print out how long it took
end_time = timer()
print(f"[INFO] Total training time: {end_time-start_time:.3f} seconds")

# Save the model
save_model(model=model_0,
           target_dir="models",
           model_name="05_going_modular_cell_mode_tinyvgg_model.pth")

  0%|          | 0/5 [00:00<?, ?it/s]

 batch no: 0 , loss : 1.0878868103027344
 batch no: 1 , loss : 1.14849853515625
 batch no: 2 , loss : 1.1205554008483887
 batch no: 3 , loss : 1.254094123840332
 batch no: 4 , loss : 1.0154976844787598
 batch no: 5 , loss : 1.0442390441894531
 batch no: 6 , loss : 1.2605568170547485
 batch no: 7 , loss : 1.0331975221633911
 batch no: 8 , loss : 1.0288009643554688
 batch no: 9 , loss : 1.0157761573791504
 batch no: 10 , loss : 1.0447596311569214
 batch no: 11 , loss : 1.2964699268341064
 batch no: 12 , loss : 1.0451260805130005
 batch no: 13 , loss : 1.0416311025619507
 batch no: 14 , loss : 1.308275580406189
 batch no: 15 , loss : 1.0173298120498657
 batch no: 16 , loss : 1.0062799453735352
 batch no: 17 , loss : 0.9846950173377991
 batch no: 18 , loss : 0.9696749448776245
 batch no: 19 , loss : 1.0724769830703735
 batch no: 20 , loss : 1.083649754524231
 batch no: 21 , loss : 0.8943887948989868
 batch no: 22 , loss : 0.8795575499534607
 batch no: 23 , loss : 1.4201347827911377
 batch 

### 6.1 Train, evaluate and save the model (script mode)

Let's combine all of our modular files into a single script `train.py`.

This will allow us to run all of the functions we've written with a single line of code on the command line:

`python going_modular/train.py`

Or if we're running it in a notebook:

`!python going_modular/train.py`

We'll go through the following steps:
1. Import the various dependencies, namely `torch`, `os`, `torchvision.transforms` and all of the scripts from the `going_modular` directory, `data_setup`, `engine`, `model_builder`, `utils`.
  * **Note:** Since `train.py` will be *inside* the `going_modular` directory, we can import the other modules via `import ...` rather than `from going_modular import ...`.
2. Setup various hyperparameters such as batch size, number of epochs, learning rate and number of hidden units (these could be set in the future via [Python's `argparse`](https://docs.python.org/3/library/argparse.html)).
3. Setup the training and test directories.
4. Setup device-agnostic code.
5. Create the necessary data transforms.
6. Create the DataLoaders using `data_setup.py`.
7. Create the model using `model_builder.py`.
8. Setup the loss function and optimizer.
9. Train the model using `engine.py`.
10. Save the model using `utils.py`.

In [ ]:
%%writefile going_moduler/train.py
"""
Trains a PyTorch image classification model using device-agnostic code.
"""

import os

import torch

from torchvision import transforms
from torch.optim import Adam

import data_setup, engine, model_builder, utils



# Setup hyperparameters
NUM_EPOCHS = 5
BATCH_SIZE = 32
HIDDEN_UNITS = 10
LEARNING_RATE = 0.001

# Setup directories
train_dir = "data/pizza_steak_sushi/train"
test_dir = "data/pizza_steak_sushi/test"

# Setup target device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Create transforms
data_transform = transforms.Compose([
  transforms.Resize((64, 64)),
  transforms.ToTensor()
])

# Create DataLoaders with help from data_setup.py
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=data_transform,
    batch_size=BATCH_SIZE
)

# Create model with help from model_builder.py
model = model_builder.TinyVGG(
    input_shape=3,
    hidden_units=HIDDEN_UNITS,
    output_shape=len(class_names)
).to(device)

# Set loss and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer=Adam(params=model_0.parameters(),
               lr=0.01)

# Start training with help from engine.py
engine.train(model=model,
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             epochs=NUM_EPOCHS,
             device=device)

# Save the model with help from utils.py
utils.save_model(model=model,
                 target_dir="models",
                 model_name="05_going_modular_script_mode_tinyvgg_model.pth")

Overwriting going_moduler/train.py


Now our final directory structure looks like:
```
data/
  pizza_steak_sushi/
    train/
      pizza/
        train_image_01.jpeg
        train_image_02.jpeg
        ...
      steak/
      sushi/
    test/
      pizza/
        test_image_01.jpeg
        test_image_02.jpeg
        ...
      steak/
      sushi/
going_modular/
  data_setup.py
  engine.py
  model_builder.py
  train.py
  utils.py
models/
  saved_model.pth
```

Now to put it all together!

Let's run our `train.py` file from the command line with:

```
!python going_modular/train.py
```


In [ ]:
!python going_moduler/train.py

Traceback (most recent call last):
  File "going_moduler/train.py", line 12, in <module>
    import data_setup, engine, model_builder, utils
  File "c:\Work\PyTorch\DL_with_PyTorch\going_moduler\engine.py", line 12, in <module>
    optimizer: torch.optim.optimizer,
AttributeError: module 'torch.optim' has no attribute 'optimizer'


Woah!

Look at that!

We've just trained a model with a single line of code from the command line.

We wrote a fair of code to do so, however, now we've got our code in `.py` files we can import them and reuse them as much as we like.